# Extract Envelope 

### Imports

In [6]:
from aubio import source, onset
import soundfile as sf
import numpy as np

In [3]:
def get_onsets(fname):
    '''
    extract onsets from audio file
    returns a list of frames where onsets occur in the audio file
    https://github.com/aubio/aubio/blob/master/python/demos/demo_onset.py
    '''
    win_s = 512                 # fft size
    hop_s = win_s // 2          # hop size

    samplerate = 0
    s = source(fname, samplerate, hop_s)
    samplerate = s.samplerate

    o = onset("default", win_s, hop_s, samplerate)

    # list of onsets, in samples
    onsets = []

    # total number of frames read
    total_frames = 0
    while True:
        samples, read = s()
        if o(samples):
            onsets.append(o.get_last())
        total_frames += read
        if read < hop_s: break
    return onsets

In [4]:
def attack(signal, onset, srate):
    '''
    Computes attack time by taking the difference 
    of the argmax and onset and dividing by the sampling rate
    '''
    return (np.argmax(np.abs(signal)) - onset)/srate

def extract_envelope(fname, win_size=256, hop_size=128):
    '''
    Computes attack time, decay time (time to reach sustain powerlevel),
    sustain (time and powerlevel), and release time
    
    Returns Dictionary of the following format:
    
    {
        "attack": attack_time,
        "decay": decay_time,
        "sustain": (sustain_time, sustain_power),
        "release": release_time,
    }
    '''
    first_onset = get_onsets(fname)[0]
    signal, sr = sf.read(fname)
    attack_t = attack(signal, first_onset, sr, win_size, hop_size)
    
    return {
        "attack": attack_t,
        "decay": 0.00,
        "sustain": (0.00, 0.00),
        "release": 0.00,
    }
    